# What are the details (metadata) of one of my _files_?
### Overview
Here we focus on _getting the details_ (or _metadata_) from a single file. As with any **detail**-type call, we will get extensive information about one file but must first know that files's id. We will show two options (A & B) of getting the same information.

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly here).
 4. You understand how to <a href="files_listAll.ipynb" target="_blank">list</a> files within one of your projects (we will just use that call directly here).
 5. You have added at least one TCGA file into your project (protected or open access)
 
### Imports and Definitions
 A single call is sufficient. We will show two different options, both of which are defined in the apimethods.py file.

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
We start by listing all of your files, then get more information on the first one. A **detail**-call for files returns the following *attributes*:
* **created_on** File creation date
* **id**     _Unique_ identifier for the file
* **name**   Name of the file, note this **is** metadata and can be _changed_
* **href**   Address<sup>1</sup> of the file.
* **modified_on** File modification date
* **metadata** Dictionary of metadata
* **origin**  Will link back to a *task* if this is an output file 
* **project** Project the file is in
* **size** file size in bytes
* **flag**   (unimportant) this is set by the object constructor, here always 'longList':False 

The **metadata** dictionary is changeable in user uploaded files. For TCGA files, this is **immutable** and the list is:
* vital_status
* reference_genome
* aliquot_id
* age_at_diagnosis
* sample_id
* days_to_death
* ethnicity
* sample_uuid
* sample_type
* platform
* data_subtype
* experimental_strategy
* case_uuid
* data_type
* case_id
* investigation
* aliquot_uuid
* disease_type
* gender
* data_format
* race
* primary_site

<sup>1</sup> This is the address where, by using API you can get this resource

In [ ]:
# [USER INPUT] Set project (p_) and file (f_) indices here:
p_index = 2
f_index = 0

existing_projects = API('projects')                              # LIST all projects

# list the files in the first project
my_files = API(path='files', query={'project': existing_projects.id[p_index]})

# get details of the first file inside the project
single_file = API(path=('files/' + my_files.id[f_index]))

print('You have selected file %s (size %s [bytes]). \n' % (single_file.name, single_file.size))

if hasattr(single_file, 'metadata'):
    if ('age_at_diagnosis' in single_file.metadata.keys()) \
    and ('experimental_strategy' in single_file.metadata.keys()):
        print(('Subject was %i years old at diagnosis; alliquot was processed with %s' % \
               (single_file.metadata['age_at_diagnosis'], \
                single_file.metadata['experimental_strategy'] ))) 

## (Option B) Use api_call() *function*
Using the api_call() function returns the same information as above, but now as *keys* within a dictionary for each project. To start, we list the first 25 projects you are a member of. Then, we query the _file list_ for the first project.

<sup>2</sup> Note that each entry in the 'items' list has the complete information for a single member, unlike the object above which organizes and concantenates all the dictionaries.

In [ ]:
# [USER INPUT] Set project (p_) and file (f_) indices here:
p_index = 2
f_index = 0

# LIST some (n=25) projects
existing_projects = api_call(path='projects', method='GET', \
                            query={'limit':25, 'offset':0}, data=None)

# LIST some (n=26) files in the project
my_files = api_call(path='files',\
                   query={'limit':26, 'project': existing_projects['items'][p_index]['id']})

# get (and print) details of a file inside that project
api_call(path=('files/' + my_files['items'][f_index]['id']))

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/get-file-details)